# <font color=#B40431>Ensemble Methods<font>

## <font color=#424242>A Sexy Machine Learning Strategy<font>

### Dataset

We want to assess the credit risk of someone using its finantial history. 
Basically, we'll use 20 finantial attributes to classify people as good or bad credit risks. 

We have a set of samples labeled as low risk or high risk. So, this is a binary classification problem.

Attributes:
-  __A1:__ status of existing checking account (categorical)
-  __A2:__ credit duration in months (numerical)
-  __A3:__ credit history (categorical)
-  __A4:__ credit purpose (categorical)
-  __A5:__ credit amount (numerical)
-  __A6:__ savings account/bonds (categorical)
-  __A7:__ how long one is employed in the current job (categorical)
-  __A8:__ installment rate in percentage of disposable income (numerical)
-  __A9:__ personal status and sex (categorical)
-  __A10:__ other debtors/guarantors (categorical)
-  __A11:__ how long one lives in the current residence (numerical)
-  __A12:__ property (categorical)
-  __A13:__ age (numerical)
-  __A14:__ other installment plans (categorical)
-  __A15:__ housing (categorical)
-  __A16:__ number of existing credits at this bank (numerical)
-  __A17:__ job (categorical)
-  __A18:__ number of people being liable to provide maintenance for (numerical) 
-  __A19:__ telephone (categorical, bool)
-  __A20:__ foreign worker (categorical, bool)

The dataset is available online:
https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/

## Settings

In [ ]:
# imports
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# seeding the generated number makes our results reproducible
from random import seed 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_recall_curve, roc_curve, auc

In [ ]:
# directory and paths
project_dir = os.path.dirname(os.getcwd())
path_data = os.path.join(project_dir, 'data', 'german_data.txt')

settings = {'grid_search':False}
seed(1994)

## Load Data

In [ ]:
# load dataset
column_name = ['A'+str(i) for i in range(1,21)] + ['TARGET']
df_data = pd.read_csv(path_data, sep=" ", header=None, names=column_name, dtype={'TARGET':str})

# replace class names
df_data.replace({'TARGET':{'1':'good', '2':'bad'}}, inplace=True)
df_data.head()

# Exploratory Data Analysis

*Use the Scikit-learn flowchart to find one reliable estimator for the job
https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html

In [ ]:
# 1 - check how many samples you have
print('Number of samples: {}'.format(df_data.shape[0]))

In [ ]:
# 2 - check if classes are balanced or unbalanced
data = df_data.TARGET.value_counts()
#sns.set(style="whitegrid")

plt.figure(figsize=(8, 6))
sns.set()
sns.barplot(x = data.index, y=data.values, palette="Blues_d", order=['good', 'bad'])
plt.title('Classes Distribution')
plt.xlabel('Classes')
plt.ylabel('Frequency')
plt.show()

Comment: Classes are unbalanced.

In [ ]:
# 3 - check a numerical feature - credit duration in month
plt.figure(figsize=(8, 6))
sns.set()
sns.boxplot(x="TARGET", y="A2", data=df_data, palette="Blues_d", whis=10)
plt.title('Credit Duration')
plt.xlabel('Classes')
plt.ylabel('Duration')
plt.show()

In [ ]:
# 4 - check a category feature - status of existing checking accounts
data = pd.DataFrame(df_data.groupby('TARGET').apply(lambda x: x['A1'].value_counts()))
data.reset_index(inplace=True)
data.columns=['TARGET', 'A1', 'COUNT']

plt.figure(figsize=(10, 6))
sns.set()
sns.barplot(x='A1', y='COUNT', hue='TARGET', data=data, palette="Blues_d", hue_order=['good', 'bad'], 
            order=sorted(data.A1.unique()))
plt.title('Checking Accounts')
plt.xlabel('Status of Existing Checking Accounts')
plt.ylabel('Frequency')
plt.show()

-  __A11:__ 0 DM
-  __A12:__ < 200 DM
-  __A13:__ >= 200 DM
-  __A14:__ no checking account

*DM stands for Deutsche Mark

In [ ]:
# 5 - check a category feature - credit history
data = pd.DataFrame(data=df_data.groupby('TARGET')['A3'].value_counts())
data.columns = ['COUNT']
data.reset_index(inplace=True)

plt.figure(figsize=(10, 6))
sns.set()
sns.barplot(x='A3', y='COUNT', hue='TARGET', data=data, palette="Blues_d", hue_order=['good', 'bad'], 
            order=sorted(data.A3.unique()))
plt.title('Credit History')
plt.xlabel('Credit history')
plt.ylabel('Frequency')
plt.show()

-  __A30:__ No credit taken/ all credits paid back duly
-  __A31:__ All credits at this bank paid back duly
-  __A32:__ Existing credits paid back duly till now
-  __A33:__ Delay in paying off in the past
-  __A34:__ Critical account/ other credits existing at different banks


### <font color=#B40431>__Exercise 1:__<font> 
-  Generate the bar chart for attribute __A17__, grouped by class. 
-  Draw conclusions from the graph.

In [ ]:
# check a category feature - job qualification
# insert your code here

-  __A171:__ unemployed or unskilled - non-residen
-  __A172:__ unskilled - resident
-  __A173:__ skilled employee or official
-  __A174:__ management, self-employed, highly qualified employee or officer

## Convert categorical features into dummies

In [ ]:
# get predictors and classes
inputs = df_data.iloc[:,:-1]
targets = df_data['TARGET']

# Convert categorical variable into dummy/indicator variables
categ_cols = ['A1', 'A3', 'A4', 'A6', 'A7', 'A9', 'A10', 'A12', 'A14', 'A15', 'A17', 'A19', 'A20']
inputs = pd.get_dummies(inputs, columns=categ_cols, drop_first=True)
inputs.head()

## Split data into training and test sets

In [ ]:
# Split data into random train and test subsets
X_train, X_test, y_train, y_test = train_test_split(inputs, targets, test_size=0.3, stratify=targets, random_state=40)

# <font color=#B40431>Random Forest<font>

## Grid search

In [ ]:
# Define parameters 
params_grid = {
    'n_estimators': [10, 100, 1000, 5000],
    'criterion': ['gini', 'entropy'],
    'max_depth': [None],
    'min_samples_split': [2],
    'min_samples_leaf': [1],
    'max_features': ['auto'],
    'bootstrap': [True, False]
}

In [ ]:
# Exhaustive search over specified parameter values for an estimator
if settings['grid_search']:
    clf_srch_rf = GridSearchCV(RandomForestClassifier(), params_grid, scoring='roc_auc', cv=3)
    clf_srch_rf.fit(X_train, y_train)

    # Get best parameters
    params = clf_srch_rf.best_params_
else:
    # If you want to avoid performing the grid search, uncomment the code below
    params = {
        'bootstrap': True, 
        'criterion': 'gini',
        'max_depth': None,
        'max_features': 'auto',
        'min_samples_leaf': 1,
        'min_samples_split': 2,
        'n_estimators': 1000}
params

## Model Fitting

In [ ]:
# Build a forest of trees from the training set (X_train, y_train)
clf_rf = RandomForestClassifier(**params)
clf_rf.fit(X_train, y_train)

# Predict class for X_train and X_test
y_pred_train = clf_rf.predict(X_train)
y_pred_test = clf_rf.predict(X_test)

## Performance Evaluation

\begin{equation}
TPR = Recall = \frac{TP}{P} = \frac{TP}{TP+FN}
\end{equation}

\begin{equation}
FPR = Fall-out = \frac{FP}{N} = \frac{FP}{FP+TN}
\end{equation}

\begin{equation}
PPV = Precision = \frac{TP}{TP+FP}
\end{equation}

\begin{equation}
f1 = 2\frac{PPV*TPR}{PPV+TPV}
\end{equation}

In [ ]:
# Compute confusion matrix to evaluate the accuracy of a classification
conf_matrix = confusion_matrix(y_test, y_pred_test, labels=['good', 'bad'])

plt.figure(figsize=(10, 6))
sns.set()
sns.heatmap(conf_matrix, xticklabels=['good', 'bad'], yticklabels=['good', 'bad'], annot=True, cmap="Blues")
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion matrix of the classifier')

plt.show()

*The label "bad" is considered as positive

In [ ]:
print('True Negative (TN): {}'.format(conf_matrix[0,0]))
print('False Positive (FP): {}'.format(conf_matrix[0,1]))
print('False Negative (FN): {}'.format(conf_matrix[1,0]))
print('True Negative (TN): {}'.format(conf_matrix[1,1]))

In [ ]:
# Get classification report
df_class_report = pd.DataFrame(classification_report(y_test, y_pred_test, output_dict=True))
print(df_class_report)

# Get accuracy score
acc = accuracy_score(y_test, y_pred_test)
print('\nAccuracy = {:.2%}'.format(acc))

In [ ]:
def binarize(y, target='bad'):
    return [1 if y_i=='bad' else 0 for y_i in y]

In [ ]:
# Plot the ROC curve 
y_pred_test_prob = clf_rf.predict_proba(X_test)
fpr, tpr, trs = roc_curve(binarize(y_test), y_pred_test_prob[:,0])

plt.figure(figsize=(8, 6))
sns.set()
sns.lineplot(fpr, tpr, palette="Blues_d")
plt.title('ROC Curve')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.show()

print('AUC: {:.2}'.format(auc(fpr, tpr)))

### <font color=#B40431>__Exercise 2:__<font> 
-  Check for overfitting by computing the accuracy score both for training set and test set.
-  As you know, RF has some knobs to avoid overfitting. Apply some changes to such parameters and draw some conclusions.

In [ ]:
# Check for overfitting
# if accuracy score for the training set is much higher than the accuracy for the test set, 
# the model is prone to overfitting

# training set
# insert your code here

# test set
# insert your code here

### <font color=#B40431>__Exercise 3:__<font> 
-  Check the feature importance. Find out which are the 5 attributes with higher importance.

In [ ]:
# Return the feature importances 
# insert your code here

# <font color=#B40431>Gradient Boosting Machine<font>

### <font color=#B40431>__Exercise 4:__<font> 
-  Fit a Gradient Boosting Machine to the German Credit Risk Assessment dataset.
-  Evaluate the model performance.
-  Check which model performs better in this case and try to understand why.
    
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html

## Grid search

In [ ]:
# Define parameters 
params_grid = {
    'loss': ['deviance', 'exponential'],
    'learning_rate': [0.1],
    'n_estimators': [10, 100, 1000, 5000],
    'criterion': ['friedman_mse'],
    'max_depth': [None],
    'min_samples_split': [2],
    'min_samples_leaf': [1],
    'max_features': ['auto'],
    'subsample': [0.5]
}

In [ ]:
# Exhaustive search over specified parameter values for an estimator
if settings['grid_search']:
    # insert your code here
    pass
else:
    # If you want to avoid performing the grid search, uncomment the code below
    params = {
        'criterion': 'friedman_mse',
        'learning_rate': 0.05,
        'max_depth': 30,
        'max_features': 'auto',
        'min_samples_leaf': 5,
        'min_samples_split': 10,
        'n_estimators': 350,
        'subsample': .6
        }
params

## Model Fitting

In [ ]:
# Build a forest of trees from the training set (X_train, y_train)
# insert your code here

# Predict class for X_train and X_test
# insert your code here

## Performance Evaluation

In [ ]:
# Compute confusion matrix to evaluate the accuracy of a classification
# insert your code here

In [ ]:
# Print TN, FP, FN and TN
# insert your code here

In [ ]:
# Get classification report
# insert your code here

# Get accuracy score
# insert your code here

In [ ]:
# Plot the ROC curve 
# insert your code here

In [ ]:
# Check for overfitting
# if accuracy score for the training set is much higher than the accuracy for the test set, 
# the model is prone to overfitting

# training set
# insert your code here

# test set
# insert your code here

In [ ]:
# Return the feature importances
# insert your code here